In [ ]:
import threading
import cv2
import tensorflow as tf
from ultralytics import YOLO
from datetime import datetime
import os

# Load models
yamnet_model = tf.keras.models.load_model("best_model.keras")  # Replace with the actual YAMNet model path
yolo_model = YOLO('yolo_v1 (2).pt')  # Replace with your YOLO model path

# Ensure the 'anomalous_videos' directory exists
os.makedirs("anomalous_videos", exist_ok=True)

def process_video_stream(video_source=0):
    cap = cv2.VideoCapture(video_source)  # Use 0 for the default webcam
    if not cap.isOpened():
        print("Error: Could not open video stream")
        return

    # VideoWriter settings
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    out = None
    recording = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to capture frame, skipping...")
            continue

        # YOLO Prediction
        results = yolo_model.predict(frame, conf=0.5, verbose=False)
        annotated_frame = results[0].plot()

        # Check for anomalies
        anomalies_detected = False
        if results[0].boxes:
            for box in results[0].boxes:
                cls_id = int(box.cls)
                confidence = box.conf.item()
                label = yolo_model.names[cls_id]

                print(f"[{datetime.now()}] Detected: {label} (Confidence: {confidence:.2f})")
                if label in ["violence", "weaponized"]:
                    anomalies_detected = True

                    if not recording:
                        video_filename = f"anomalous_videos/{label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
                        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
                        recording = True
                        print(f"[{datetime.now()}] Recording started: {video_filename}")
                    
                    if out:
                        out.write(frame)
                    break

        # Stop recording if no anomalies
        if not anomalies_detected and recording:
            print(f"[{datetime.now()}] Anomaly ended. Stopping recording.")
            recording = False
            if out:
                out.release()
                out = None

        # Optional: Display annotated video stream
        # cv2.imshow("Video Stream", annotated_frame)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit requested. Stopping program.")
            break

    # Release resources
    if recording and out:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

# Main function
if __name__ == "__main__":
    video_thread = threading.Thread(target=process_video_stream, args=(0,))
    video_thread.start()
    video_thread.join()


[2025-01-10 18:19:55.703385] Detected: non-violence (Confidence: 0.93)
[2025-01-10 18:19:55.819418] Detected: violence (Confidence: 0.92)
[2025-01-10 18:19:55.820086] Recording started: anomalous_videos/violence_20250110_181955.mp4
[2025-01-10 18:19:55.903324] Detected: violence (Confidence: 0.92)
[2025-01-10 18:19:55.999811] Detected: violence (Confidence: 0.95)
[2025-01-10 18:19:56.089110] Detected: violence (Confidence: 0.91)
[2025-01-10 18:19:56.169620] Detected: violence (Confidence: 0.94)
[2025-01-10 18:19:56.253637] Detected: violence (Confidence: 0.93)
[2025-01-10 18:19:56.336117] Detected: violence (Confidence: 0.93)
[2025-01-10 18:19:56.405396] Detected: violence (Confidence: 0.93)
[2025-01-10 18:19:56.488762] Detected: violence (Confidence: 0.95)
[2025-01-10 18:19:56.569224] Detected: violence (Confidence: 0.95)
[2025-01-10 18:19:56.638217] Detected: violence (Confidence: 0.93)
[2025-01-10 18:19:56.735210] Detected: violence (Confidence: 0.95)
[2025-01-10 18:19:56.802218] De